In [1]:
import findspark
findspark.init()
import shapely
import pandas as pd
import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from shapely import wkt

In [2]:
spark = SparkSession. \
builder. \
appName('appName'). \
config("spark.serializer", KryoSerializer.getName). \
config("spark.executor.memory", "5g"). \
config("spark.driver.memory", "10g"). \
config('spark.driver.maxResultSize', '5g'). \
config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.2.0-incubating,org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
getOrCreate()
SedonaRegistrator.registerAll(spark)

True

In [3]:
BD_MANZANAS = spark.read.parquet(f"../SCINCE_Parquets/*.parquet")
BD_MANZANAS_EEVVV = BD_MANZANAS.select('CVEGEO', 'geometry')
BD_MANZANAS_EEVVV.cache()
BD_MANZANAS_EEVVV.printSchema()
BD_MANZANAS_EEVVV.show()

root
 |-- CVEGEO: string (nullable = true)
 |-- geometry: string (nullable = true)

+----------------+--------------------+
|          CVEGEO|            geometry|
+----------------+--------------------+
|1900100010021028|POLYGON ((2659951...|
|1900100010021031|POLYGON ((2659516...|
|1900100010021036|POLYGON ((2659395...|
|1900100010036005|POLYGON ((2660345...|
|1900100010036019|POLYGON ((2660117...|
|1900100010036028|POLYGON ((2660120...|
|1900100010036029|POLYGON ((2660279...|
|1900100010036030|POLYGON ((2660352...|
|1900100010036042|POLYGON ((2660087...|
|1900200010207004|POLYGON ((2745115...|
|1900200010207010|POLYGON ((2744991...|
|1900200010207023|POLYGON ((2745358...|
|1900200010207035|POLYGON ((2745284...|
|1900200010207040|POLYGON ((2745279...|
|1900200010211009|POLYGON ((2744795...|
|1900200010211014|POLYGON ((2744493...|
|1900200010211016|POLYGON ((2744699...|
|1900200010211019|POLYGON ((2744975...|
|1900200010211024|POLYGON ((2744589...|
|1900200010211028|POLYGON ((2744969.

In [4]:
BD_MANZANAS_EEVVV.createOrReplaceTempView("manzanas")



In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import pyproj
gas = spark.read.csv('gasolineras_rating.csv', header=True, inferSchema=True)

# Define el CRS de entrada (WGS 84) y el CRS de salida (por ejemplo, UTM Zone 14N)
crs_entrada = pyproj.CRS("EPSG:4326")  # WGS 84 (Latitud/Longitud)
crs_salida = 'PROJCS["Mexico_ITRF2008_LCC",GEOGCS["Mexico_ITRF2008",DATUM["Mexico_ITRF2008",SPHEROID["GRS_1980",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP",AUTHORITY["EPSG","9802"]],PARAMETER["Central_Meridian",-102],PARAMETER["Latitude_Of_Origin",12],PARAMETER["False_Easting",2500000],PARAMETER["False_Northing",0],PARAMETER["Standard_Parallel_1",17.5],PARAMETER["Standard_Parallel_2",29.5],PARAMETER["Scale_Factor",1],UNIT["Meter",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","6372"]]'

# Crea un transformador de coordenadas
transformador = pyproj.Transformer.from_crs(crs_entrada, crs_salida, always_xy=True)

# Define una función UDF (User Defined Function) para aplicar la transformación
def transformar_coordenadas(latitude, longitude):
    x, y = transformador.transform(longitude, latitude)
    return f"POINT ({x:.8f} {y:.8f})"

# Agrega una nueva columna 'geometry' con las coordenadas transformadas
gas = gas.withColumn("geometry", F.udf(transformar_coordenadas)(col("latitude"), col("longitude")))
from pyspark.sql.functions import expr

# Convierte la columna 'geometry' en objetos geométricos
gas = gas.withColumn("geometry", expr("ST_GeomFromText(geometry)"))
# Muestra el DataFrame resultante
gas.show()

# Registrar el DataFrame de Spark como una vista temporal en Spark SQL
gas.createOrReplaceTempView("gas")

+--------------------+----------+------------+----------------------+--------------------+
|           google_id|  latitude|   longitude|Average of CALIF TOTAL|            geometry|
+--------------------+----------+------------+----------------------+--------------------+
|0x8662bfb106713cc...| 25.644376|-100.2971011|           8.301587302|POINT (2670178.67...|
|0x8662be01437db22...|  25.66811|-100.3450231|           8.227272727|POINT (2665359.62...|
|0x8662957f9c8379d...| 25.678379| -100.303761|           7.613095238|POINT (2669468.81...|
|0x8662bf41d52a8ae...|25.6490203|-100.2742219|           7.547794118|POINT (2672458.83...|
|0x8662969404a8738...|25.7371591|-100.3672212|                   7.5|POINT (2663055.09...|
|0x8662971d5f7b572...| 25.728984|-100.3947993|           7.488461538|POINT (2660311.25...|
|0x866296756b0d543...| 25.697366| -100.351063|           7.417249417|POINT (2664719.08...|
|0x8662979ce484b33...|25.7136447|-100.3632576|           6.794871795|POINT (2663480.48...|

In [ ]:
x

In [8]:
Influencia_reviews_rating = spark.sql("""SELECT*
FROM gas
""").toPandas()

Influencia_reviews_shp = gpd.GeoDataFrame(Influencia_reviews_rating, geometry="geometry")
archivo = 'Influencia_reviews_rating'
Influencia_reviews_shp.to_file(f"./resultados/{archivo}.shp")
crs='PROJCS["Mexico_ITRF2008_LCC",GEOGCS["Mexico_ITRF2008",DATUM["Mexico_ITRF2008",SPHEROID["GRS_1980",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP",AUTHORITY["EPSG","9802"]],PARAMETER["Central_Meridian",-102],PARAMETER["Latitude_Of_Origin",12],PARAMETER["False_Easting",2500000],PARAMETER["False_Northing",0],PARAMETER["Standard_Parallel_1",17.5],PARAMETER["Standard_Parallel_2",29.5],PARAMETER["Scale_Factor",1],UNIT["Meter",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","6372"]]'
PRJ_file = open(f"./resultados/{archivo}.prj","w")
PRJ_file.write(crs)
PRJ_file.close()

C:\Users\grise\AppData\Local\Temp\ipykernel_25788\4123383362.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  Influencia_reviews_shp.to_file(f"./resultados/{archivo}.shp")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import pyproj
gas = spark.read.csv('Outscraper-202309280607358c12_gasolinera.csv', header=True, inferSchema=True)

# Define el CRS de entrada (WGS 84) y el CRS de salida (por ejemplo, UTM Zone 14N)
crs_entrada = pyproj.CRS("EPSG:4326")  # WGS 84 (Latitud/Longitud)
crs_salida = 'PROJCS["Mexico_ITRF2008_LCC",GEOGCS["Mexico_ITRF2008",DATUM["Mexico_ITRF2008",SPHEROID["GRS_1980",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP",AUTHORITY["EPSG","9802"]],PARAMETER["Central_Meridian",-102],PARAMETER["Latitude_Of_Origin",12],PARAMETER["False_Easting",2500000],PARAMETER["False_Northing",0],PARAMETER["Standard_Parallel_1",17.5],PARAMETER["Standard_Parallel_2",29.5],PARAMETER["Scale_Factor",1],UNIT["Meter",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","6372"]]'

# Crea un transformador de coordenadas
transformador = pyproj.Transformer.from_crs(crs_entrada, crs_salida, always_xy=True)

# Define una función UDF (User Defined Function) para aplicar la transformación
def transformar_coordenadas(latitude, longitude):
    x, y = transformador.transform(longitude, latitude)
    return f"POINT ({x:.8f} {y:.8f})"

# Agrega una nueva columna 'geometry' con las coordenadas transformadas
gas = gas.withColumn("geometry", F.udf(transformar_coordenadas)(col("latitude"), col("longitude")))
from pyspark.sql.functions import expr

# Convierte la columna 'geometry' en objetos geométricos
gas = gas.withColumn("geometry", expr("ST_GeomFromText(geometry)"))
# Muestra el DataFrame resultante
gas.show()

# Registrar el DataFrame de Spark como una vista temporal en Spark SQL
gas.createOrReplaceTempView("gas")

In [ ]:
BD_MANZANAS_EEVVV = BD_MANZANAS_EEVVV.withColumn('geometry', expr("ST_GeomFromWKT(geometry)"))
BD_MANZANAS_EEVVV.createOrReplaceTempView("manzanas")

In [ ]:
gas.show()

In [ ]:
Influencia_reviews = spark.sql("""SELECT 
    CVEGEO,
    manzanas.geometry,
    gas.google_id,
    gas.latitude,
    gas.longitude,
    COUNT(*) as n_reviews
FROM manzanas, gas
    WHERE (ST_Intersects(manzanas.geometry, ST_Buffer(gas.geometry, 300)))
    GROUP BY CVEGEO,manzanas.geometry,    gas.latitude,
    gas.longitude,google_id
""").toPandas()

Influencia_reviews.to_excel('reviews_cvgeo.xlsx')
Influencia_reviews_shp = gpd.GeoDataFrame(Influencia_reviews, geometry="geometry")
archivo = 'manzanas_kmeans_gasolineras_reviews'
Influencia_reviews_shp.to_file(f"./resultados/{archivo}.shp")
crs='PROJCS["Mexico_ITRF2008_LCC",GEOGCS["Mexico_ITRF2008",DATUM["Mexico_ITRF2008",SPHEROID["GRS_1980",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP",AUTHORITY["EPSG","9802"]],PARAMETER["Central_Meridian",-102],PARAMETER["Latitude_Of_Origin",12],PARAMETER["False_Easting",2500000],PARAMETER["False_Northing",0],PARAMETER["Standard_Parallel_1",17.5],PARAMETER["Standard_Parallel_2",29.5],PARAMETER["Scale_Factor",1],UNIT["Meter",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","6372"]]'
PRJ_file = open(f"./resultados/{archivo}.prj","w")
PRJ_file.write(crs)
PRJ_file.close()

In [ ]:
Influencia_reviews

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Ruta al archivo shapefile generado
archivo = 'manzanas_kmeans_gasolineras_reviews'
ruta_shapefile = f"./resultados/{archivo}.shp"

# Cargar el archivo shapefile con geopandas
data = gpd.read_file(ruta_shapefile)

# Mostrar el mapa
data.plot(figsize=(10, 10), cmap='viridis')
plt.title('Mapa de influencia de gasolineras en manzanas')
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Ruta al archivo shapefile
archivo_manzanas = 'manzanas_kmeans_gasolineras_reviews'
ruta_manzanas = f"./resultados/{archivo_manzanas}.shp"

# Cargar el archivo shapefile con geopandas
manzanas = gpd.read_file(ruta_manzanas)

# Graficar el mapa
fig, ax = plt.subplots(figsize=(10, 10))

# Graficar las manzanas
manzanas.plot(ax=ax, color='blue', alpha=0.5)

# Agregar círculos para señalar áreas de manzanas
for idx, row in manzanas.iterrows():
    # Obtener el centroide de cada geometría de manzana
    centroid = row['geometry'].centroid
    
    # Dibujar un círculo con el centro en el centroide de la manzana
    circle = plt.Circle((centroid.x, centroid.y), 20, color='red', fill=False)
    ax.add_patch(circle)

plt.title('Mapa de manzanas con círculos señalando áreas')
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

Influencia_reviews = spark.sql("""
    SELECT 
        CVEGEO,
        manzanas.geometry,
        ST_Centroid(ST_Intersection(manzanas.geometry, ST_Buffer(gas.geometry, 1500))) as centroid
    FROM manzanas, gas
    WHERE ST_Intersects(manzanas.geometry, ST_Buffer(gas.geometry, 1500))
""").toPandas()

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Cargar el resultado desde Spark a un DataFrame de Pandas
resultados_pandas = Influencia_reviews

# Convertir el DataFrame de Pandas a un GeoDataFrame de GeoPandas
gdf = gpd.GeoDataFrame(resultados_pandas, geometry='centroid')

# Graficar el resultado
fig, ax = plt.subplots(figsize=(10, 10))

# Graficar las manzanas
manzanas.plot(ax=ax, color='blue', alpha=0.5)

gdf['geometry'] = gpd.GeoSeries(gdf['geometry'])

# Crear buffers para los puntos de gas
gdf['buffer'] = gdf['geometry'].buffer(100)

# Graficar los buffers
gdf['buffer'].plot(ax=ax, color='orange', alpha=0.01)

# Graficar el centroide de la intersección
# gdf.plot(ax=ax, color='red', markersize=10, marker='o')

plt.title('INFLUENCIA DE GASOLINERAS ESTUDIADAS EN EL CENTRO DE MONTERREY')
plt.show()